# Impact Mechanism: Inverse Scaling Calculations

This notebook demonstrates how to use the `impact_scaling.py` module to compute impact parameters from observed crater dimensions.

## Theory

The code implements **pi-group scaling** relationships developed by Holsapple, Schmidt, and others:

### Key Dimensionless Parameters

1. **π₂ = ga/U²** - Gravity parameter (ratio of gravitational to kinetic energy)
2. **π₃ = Y/(ρU²)** - Strength parameter (ratio of material strength to dynamic pressure)
3. **π₄ = ρ_target/ρ_impactor** - Density ratio

### Scaling Regimes

- **Strength Regime**: π₃ > π₂ (small, fast impacts in strong materials)
- **Gravity Regime**: π₂ > π₃ (large impacts where gravity dominates)

### Crater Volume Scaling

$$V = K_1 \frac{m}{\rho_{target}} \pi_2^{-3\nu} \pi_3^{-3\mu} \pi_4^{\alpha}$$

Where:
- V = transient crater volume
- m = impactor mass
- ν ≈ 0.4 (gravity scaling exponent)
- μ ≈ 0.41-0.55 (strength scaling exponent)
- α ≈ 1/3 (density scaling exponent)

## References

- Holsapple, K.A. (1993). "The scaling of impact processes in planetary sciences." *Annual Review of Earth and Planetary Sciences*, 21, 333-373.
- Holsapple, K.A., & Housen, K.R. (2007). "A crater and its ejecta: An interpretation of Deep Impact." *Icarus*, 187, 345-356.
- Melosh, H.J. (1989). *Impact Cratering: A Geologic Process.* Oxford University Press.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from impact_scaling import (
    ImpactScaling, 
    MATERIALS, 
    IMPACTORS,
    print_materials,
    format_results
)

# Configure plotting
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

## 1. View Available Materials

First, let's see what target materials and impactor types are available:

In [ ]:
print_materials()

## 2. Example: Small Fresh Lunar Crater

Let's compute the impactor parameters for a small, fresh lunar crater:
- Diameter: 100 m
- Depth: 20 m (d/D = 0.2, typical for fresh simple crater)
- Impact velocity: 15 km/s (typical for lunar impacts)

In [ ]:
# Set up target and impactor
target = MATERIALS['lunar_regolith']
impactor = IMPACTORS['asteroid_rock']

# Create calculator
calc = ImpactScaling(target, impactor.density)

# Compute impactor parameters
results = calc.compute_impactor_params(
    D=100,      # Crater diameter (m)
    d=20,       # Crater depth (m)
    U=15000,    # Impact velocity (m/s)
    crater_type='simple'
)

# Display results
format_results(results, 100, 20, 15000, target.name, impactor.name)

## 3. Example: Large Complex Crater

For larger craters (typically > 15-20 km on Moon), craters become complex with:
- Central peaks or peak rings
- Terraced walls
- Lower depth/diameter ratios (d/D ≈ 0.1)
- Significant collapse from transient cavity

In [ ]:
# Large lunar crater in mare basalt
target = MATERIALS['lunar_mare']
impactor = IMPACTORS['asteroid_rock']

calc = ImpactScaling(target, impactor.density)

results_large = calc.compute_impactor_params(
    D=10000,    # 10 km diameter
    d=1000,     # 1 km depth (d/D = 0.1)
    U=20000,    # 20 km/s
    crater_type='complex'
)

format_results(results_large, 10000, 1000, 20000, target.name, impactor.name)

## 4. Velocity Trade-off Study

Since impact velocity is often uncertain, we can explore the trade-off between impactor size and velocity for a given crater. Smaller, faster impactors produce the same crater as larger, slower ones.

In [ ]:
# Study a 500 m diameter crater
target = MATERIALS['lunar_regolith']
impactor = IMPACTORS['asteroid_rock']
calc = ImpactScaling(target, impactor.density)

scan = calc.velocity_scan(
    D=500,      # 500 m crater
    d=100,      # 100 m depth
    U_range=(5000, 30000),  # 5-30 km/s
    n_points=50,
    crater_type='simple'
)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Impactor diameter vs velocity
ax = axes[0, 0]
ax.plot(scan['velocities']/1000, scan['impactor_diameters'], 'b-', linewidth=2)
ax.set_xlabel('Impact Velocity (km/s)')
ax.set_ylabel('Impactor Diameter (m)')
ax.set_title('Impactor Size vs. Impact Velocity')
ax.grid(True, alpha=0.3)

# Impactor mass vs velocity
ax = axes[0, 1]
ax.semilogy(scan['velocities']/1000, scan['impactor_masses'], 'r-', linewidth=2)
ax.set_xlabel('Impact Velocity (km/s)')
ax.set_ylabel('Impactor Mass (kg)')
ax.set_title('Impactor Mass vs. Impact Velocity')
ax.grid(True, alpha=0.3)

# Impact energy vs velocity
ax = axes[1, 0]
ax.plot(scan['velocities']/1000, scan['impact_energies']/4.184e15, 'g-', linewidth=2)
ax.set_xlabel('Impact Velocity (km/s)')
ax.set_ylabel('Impact Energy (Mt TNT)')
ax.set_title('Impact Energy vs. Velocity')
ax.grid(True, alpha=0.3)

# Scaling regime
ax = axes[1, 1]
regime_numeric = [1 if r == 'strength' else 0 for r in scan['regimes']]
ax.fill_between(scan['velocities']/1000, 0, regime_numeric, 
                alpha=0.3, color='blue', label='Strength')
ax.fill_between(scan['velocities']/1000, regime_numeric, 1, 
                alpha=0.3, color='green', label='Gravity')
ax.set_xlabel('Impact Velocity (km/s)')
ax.set_ylabel('Dominant Regime')
ax.set_title('Scaling Regime vs. Velocity')
ax.set_ylim([0, 1])
ax.set_yticks([0.25, 0.75])
ax.set_yticklabels(['Gravity', 'Strength'])
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.savefig('velocity_scan_results.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFor a {500}m diameter crater:")
print(f"  At 10 km/s: {scan['impactor_diameters'][10]:.1f} m impactor")
print(f"  At 20 km/s: {scan['impactor_diameters'][30]:.1f} m impactor")
print(f"  At 25 km/s: {scan['impactor_diameters'][40]:.1f} m impactor")

## 5. Custom Calculation: Your Own Crater

Use this cell to compute parameters for your own crater observations:

In [ ]:
# === CUSTOMIZE THESE PARAMETERS ===

# Crater dimensions
crater_diameter = 200  # meters
crater_depth = 40      # meters

# Impact conditions
impact_velocity = 18000  # m/s (18 km/s)

# Select materials from available options
# Options: 'lunar_regolith', 'lunar_mare', 'lunar_highland', 'sandstone', 
#          'granite', 'sand', 'dry_soil', 'water_ice', etc.
target_material = 'lunar_regolith'

# Options: 'asteroid_rock', 'asteroid_metal', 'comet_ice', 'comet_ice_dense'
impactor_type = 'asteroid_rock'

# Crater type: 'simple' or 'complex'
crater_classification = 'simple'

# === END CUSTOMIZATION ===

# Run calculation
target = MATERIALS[target_material]
impactor = IMPACTORS[impactor_type]
calc = ImpactScaling(target, impactor.density)

results = calc.compute_impactor_params(
    D=crater_diameter,
    d=crater_depth,
    U=impact_velocity,
    crater_type=crater_classification
)

format_results(results, crater_diameter, crater_depth, impact_velocity,
              target.name, impactor.name)

## 6. Comparing Different Target Materials

Let's see how the same crater size in different materials requires different impactors:

In [ ]:
# Fixed crater and velocity
D_crater = 1000  # 1 km
d_crater = 200   # 200 m
U = 15000        # 15 km/s

# Test different materials
materials_to_test = ['lunar_regolith', 'lunar_mare', 'sand', 'granite', 'water_ice']

print("\n" + "="*90)
print(f"Impactor Required for {D_crater}m Crater at {U/1000} km/s in Different Materials")
print("="*90)
print(f"\n{'Material':<25} {'Density':<15} {'Strength':<15} {'Impactor Ø':<15} {'Energy (Mt)'}")
print("─"*90)

impactor = IMPACTORS['asteroid_rock']

for mat_name in materials_to_test:
    target = MATERIALS[mat_name]
    calc = ImpactScaling(target, impactor.density)
    
    res = calc.compute_impactor_params(D_crater, d_crater, U, crater_type='simple')
    
    print(f"{target.name:<25} {target.density:<15.0f} {target.strength:<15.1e} "
          f"{res['impactor_diameter']:<15.2f} {res['impact_energy']/4.184e15:<.3f}")

print("\nNote: Weaker materials (lower strength) require smaller impactors for same crater.")

## 7. Scaling Law Visualization

Visualize how π₂ and π₃ control the scaling regime:

In [ ]:
# Create a parameter space plot
target = MATERIALS['lunar_regolith']
impactor = IMPACTORS['asteroid_rock']
calc = ImpactScaling(target, impactor.density)

# Vary impactor size and velocity
a_values = np.logspace(-1, 3, 50)  # 0.1 m to 1 km radius
U_values = np.logspace(3, 5, 50)   # 1 km/s to 100 km/s

A, U_grid = np.meshgrid(a_values, U_values)
pi2_grid = np.zeros_like(A)
pi3_grid = np.zeros_like(A)

for i in range(len(U_values)):
    for j in range(len(a_values)):
        pi2_grid[i, j] = calc.pi2(a_values[j], U_values[i])
        pi3_grid[i, j] = calc.pi3(U_values[i])

# Plot
fig, ax = plt.subplots(figsize=(12, 8))

# Regime boundary: π₂ = π₃
contour = ax.contour(A, U_grid/1000, np.log10(pi2_grid/pi3_grid), 
                     levels=[0], colors='red', linewidths=3)
ax.clabel(contour, inline=True, fontsize=10, fmt='π₂=π₃')

# Fill regions
ax.contourf(A, U_grid/1000, np.log10(pi2_grid/pi3_grid), 
            levels=[-10, 0, 10], colors=['lightblue', 'lightgreen'], alpha=0.5)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Impactor Radius (m)', fontsize=12)
ax.set_ylabel('Impact Velocity (km/s)', fontsize=12)
ax.set_title('Impact Scaling Regime Map\n(Blue = Strength-dominated, Green = Gravity-dominated)', 
             fontsize=14)
ax.grid(True, alpha=0.3)

# Add text labels
ax.text(0.5, 80, 'STRENGTH\nREGIME', fontsize=14, ha='center', 
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
ax.text(500, 5, 'GRAVITY\nREGIME', fontsize=14, ha='center',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

plt.tight_layout()
plt.savefig('scaling_regime_map.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Advanced: Defining Custom Materials

You can define your own custom materials:

In [ ]:
from impact_scaling import Material

# Define a custom target material
my_custom_material = Material(
    name='Custom Regolith',
    density=1700,      # kg/m³
    strength=5e3,      # Pa
    gravity=1.62,      # m/s² (lunar gravity)
    K1=0.132,          # Coupling parameter
    mu=0.41,           # Strength scaling exponent (granular)
    nu=0.40,           # Gravity scaling exponent
)

# Use custom material
calc_custom = ImpactScaling(my_custom_material, impactor_density=2700)

results_custom = calc_custom.compute_impactor_params(
    D=300, d=60, U=12000, crater_type='simple'
)

format_results(results_custom, 300, 60, 12000, 
              my_custom_material.name, 'Rocky Asteroid')

## 9. Batch Processing: Multiple Craters

Process multiple crater measurements at once:

In [ ]:
import pandas as pd

# Example crater dataset
crater_data = pd.DataFrame({
    'crater_name': ['Crater_A', 'Crater_B', 'Crater_C', 'Crater_D'],
    'diameter_m': [150, 500, 1200, 85],
    'depth_m': [30, 100, 240, 17],
    'velocity_ms': [15000, 18000, 20000, 12000],
    'material': ['lunar_regolith', 'lunar_mare', 'lunar_mare', 'lunar_regolith'],
})

# Process all craters
results_list = []

for idx, row in crater_data.iterrows():
    target = MATERIALS[row['material']]
    impactor = IMPACTORS['asteroid_rock']
    calc = ImpactScaling(target, impactor.density)
    
    res = calc.compute_impactor_params(
        D=row['diameter_m'],
        d=row['depth_m'],
        U=row['velocity_ms'],
        crater_type='simple'
    )
    
    results_list.append({
        'crater_name': row['crater_name'],
        'impactor_diameter_m': res['impactor_diameter'],
        'impactor_mass_kg': res['impactor_mass'],
        'energy_J': res['impact_energy'],
        'energy_Mt': res['impact_energy'] / 4.184e15,
        'regime': res['regime']
    })

# Create results dataframe
results_df = pd.DataFrame(results_list)

print("\nBatch Processing Results:")
print("="*100)
print(results_df.to_string(index=False))

# Save to CSV
results_df.to_csv('impact_parameters_results.csv', index=False)
print("\nResults saved to: impact_parameters_results.csv")

## Summary

This notebook demonstrates:

1. **Basic usage** - Computing impactor parameters from crater dimensions
2. **Material selection** - Choosing from predefined target and impactor materials
3. **Velocity trade-offs** - Understanding impactor size vs. velocity relationships
4. **Regime analysis** - Identifying strength vs. gravity dominated impacts
5. **Custom materials** - Defining your own material properties
6. **Batch processing** - Analyzing multiple craters efficiently

### Key Takeaways:

- Smaller, faster impactors can create the same crater as larger, slower ones
- Material strength significantly affects required impactor size
- The transition from strength to gravity regime depends on impact scale
- Uncertainty in velocity leads to wide range of possible impactor sizes

### Next Steps:

- Apply to your own crater measurements
- Experiment with different material combinations
- Incorporate uncertainty quantification
- Compare with experimental crater data